# 문서 검색 챗봇 만들기 LangChain

In [1]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2 tiktoken

## OpenAI API Key

https://platform.openai.com/account/api-keys

In [3]:
import openai
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import os

os.environ["OPENAI_API_KEY"] = ""

In [6]:
!wget https://github.com/kairess/toy-datasets/blob/master/Demian.pdf

--2023-04-18 19:34:18--  https://github.com/kairess/toy-datasets/blob/master/Demian.pdf
github.com (github.com) 해석 중... 20.200.245.247
다음으로 연결 중: github.com (github.com)|20.200.245.247|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 지정하지 않음 [text/html]
저장 위치: `Demian.pdf'

Demian.pdf              [ <=>                ] 141.66K  --.-KB/s    /  0.02s   

2023-04-18 19:34:19 (5.79 MB/s) - `Demian.pdf' 저장함 [145064]



## Preprocess a PDF file

In [8]:
reader = PdfReader("./Demian.pdf")

raw_text = ""

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

raw_text[:1000]

"DEMIAN • \nDownloaded from https://www.holybooks.comHERMANN \nHESSE • DEMIAN \n* \nTranslated by W. J. Strachan \nLondon \nDownloaded from https://www.holybooks.comPrologue \nI cannot tell my story without going a long way back. \nIf it were possible I would go back much farther still to \nthe very earliest years of my childhood and beyond them \nto my family origins. \nWhen poets write novels they are apt to behave as if \nthey were gods, with the power to look beyond and com\xad\nprehend any human story and serve it up as if the \nAlmighty himself, omnipresent, were relating it in all \nits naked truth. That I am no more able to do than the \npoets. But my story is more important to me than any \npoet's story to him, for it is my own-and it is the story \nof a huffian being-not an invented, idealised person \nbut a real, live, uniq:-e being. What constitutes a real, \nlive human being is more of a mystery than ever these \ndays, and men-each one of whom is a valuable, unique \nexper

## Summarize 요약

In [4]:
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain
# https://github.com/hwchase17/langchain/issues/1368
from langchain.llms import OpenAIChat
from langchain.chat_models import ChatOpenAI

llmc = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")

llm = OpenAI(temperature=0)

summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

summary_chain_cgpt35 = load_summarize_chain(llmc, chain_type="map_reduce")
summarize_document_chain_cgpt35 = AnalyzeDocumentChain(combine_docs_chain=summary_chain_cgpt35)

/Users/jaden/miniforge3/envs/autogpt/lib/python3.9/site-packages/langchain/llms/openai.py:661: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [7]:
from pprint import pprint
"""
OpenAI 의 Whisper 모델로 STT 결과를 텍스트 파일로 저장한 후, 이를 읽어서 사용
"""
with open('./진쏠미_transcript.txt', 'r') as f:
    transcript = f.read()
# transcript = transcript.replace('. ', '. \n')
pprint(transcript)

('여기 두 존재가 있습니다. 하나는 인공지능이고요. 하나는 인간입니다. 겉으로 보기에는 완전히 똑같은 두 존재의 정체성을 가르는 것은 단 '
 '한 가지입니다. 그것은 바로 생각하고 감정을 느낄 수 있는 힘. 의식입니다. 우리는 이 의식이라는 게 그냥 뇌라는 장기에서 뚝딱 만들어진 '
 '무산물이라고 배워왔습니다. 하지만 최첨단 현대 물리학계에서 기존의 독념을 완전히 뒤집어버리는 관념이 제기되고 있습니다. 현재 생존해 있는 '
 '과학자들 중에서 가장 위대한 물리학자로 칭송받고 있는 과학자 2020년 노벨 물리학상을 수상한 로저 펜 로즈는 이렇게 말했습니다. 인간의 '
 '의식은 뇌라는 기계장치 따위에서는 만들어질 수 없는 그 무언가이다. 만약 어떤 사람이 일시적으로 사망하게 되면 그 사람의 두뇌 속 '
 '미세소관이라는 부분에 있던 양자 정보가 우주로 흘러들어가게 된다. 그러나 죽음 근처까지 갔던 사람이 다시 살아난다면 두뇌에서 유출된 양자 '
 '정보가 다시 두뇌 속으로 돌아가게 된다. 이때가 바로 임사체험을 하게 되는 경우다. 만약 그가 되살아나지 않고 완전히 죽게 되더라도 그의 '
 '두뇌 속에 있던 양자 정보는 영혼으로써 육체 바깥에서 영원히 존재할 수 있다. 왜냐? 양자 정보는 절대로 파괴될 수 없기 때문이다. 이게 '
 '도대체 무슨 말인지 아리가리 하신 분들을 위해서 양자 의식 이론의 핵심만 간단하게 요약해보겠습니다. 인간의 의식은 양자 물질로 구성되어 '
 '있기 때문에 양자 에너지적 성질을 가진다. 의식은 양자 에너지의 형태로 두뇌 속에 미세소관이라는 부분에 담겨 있다가 육체가 죽는 순간 몸 '
 '밖으로 빠져나간다. 그렇게 육체 바깥으로 빠져나간 의식 에너지는 인간의 육체를 떠나서도 영원히 존재할 수 있다. 아니 그러니까 인류가 '
 '오래전부터 영혼이라고 불러봤던 것이 비로소 현대 물리학인 양자 역학으로도 설명이 가능하게 된 거예요. 자 의식이 양자적 성질을 가지는 '
 '에너지라면 이 의식 에너지는 존재하는 모든 에너지에 예외 없이 적

In [10]:
# 특정 llm 으로 토크나이징
import tiktoken
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
# encode the text using the GPT-3.5-Turbo encoder
tokenized_text = enc.encode(transcript)

# calculate the number of tokens in the encoded text
print(len(tokenized_text)) # 한글은 8천 토큰이고
print(len(raw_text)) # 영어는 29만 토큰

8020


In [10]:
# 헤르만 헤세 요약결과
pprint(summarize_document_chain.run(raw_text))

(" Hermann Hesse's Demian follows the journey of a young man as he searches "
 'for his purpose in life and learns to accept himself. Along the way, he '
 'meets Max Demian, who helps him interpret religious stories in a more '
 'personal way, and Alfons Beck, who encourages him to express his thoughts '
 'through poetry. He discovers his destiny is to find his own path and live it '
 'out wholly and resolutely within himself. He reunites with Demian and Frau '
 'Eva, who encourages him to find his dream and remain faithful to it. During '
 'the war, Sinclair experiences a strange feeling and is eventually reunited '
 "with Demian, who has the 'sign' on his forehead.")


In [15]:
# https://python.langchain.com/en/latest/reference/modules/text_splitter.html
##############################################################################
# 해결 : langchain의 text_splitter 를 활용해서 토큰 기반으로 문장을 나누고 -> 
# langchain.schema의 Document를 활용하여 나뉜 문장에 page_content특성을 부여 후 
# chat_models 의 ChatOpenAI 를 활용하여 요약을 진행하면 된다.
##############################################################################
from langchain import text_splitter
# from langchain.docstore.document import Document
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from pprint import pprint
'''
# Error handling
ValueError: OpenAIChat currently only supports single prompt, got [ 
openAIchat 말고 ChatOpenAI 로 바꿔야 됨
'''
paragraphs = text_splitter.TokenTextSplitter().split_text(transcript)
# pprint(paragraphs)
docs = [Document(page_content=t) for t in paragraphs]
# pprint(docs)     

llmc = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")
summary_chain_cgpt35 = load_summarize_chain(llmc, chain_type="map_reduce") # map_reduce 는, 요약은 하는데 결과가 영어로 나옴
# summary_chain_cgpt35 = load_summarize_chain(llmc, chain_type="refine") # refine 은, 결과가 안나옴 뭔가 디폴트 프롬프트가 다른것 같음

pprint(summary_chain_cgpt35.run(docs)) # 결과가 영어로 나와서 프롬프트 템플릿을 통해 한글로 나오게 해보자
# 8020 토큰에 51초 걸림

# summarize_document_chain_cgpt35 = AnalyzeDocumentChain(combine_docs_chain=summary_chain_cgpt35)

# # AnalyzeDocumentChain은 
# # TypeError: object of type 'Document' has no len()
# # 에러가 발생함

# summarize_document_chain_cgpt35 = AnalyzeDocumentChain(combine_docs_chain=summary_chain_cgpt35)
# pprint(summarize_document_chain_cgpt35.run(docs))

('The article discusses the concept of quantum consciousness, which suggests '
 'that human consciousness is made up of quantum energy and may have an '
 'eternal soul that cannot be destroyed. The principles of karma and '
 'reincarnation have been accepted by ancient civilizations and can be '
 'explained using quantum mechanics. Consciousness energy is limited and can '
 'be forgotten in real-time. The brain acts as a filter, and problems with '
 'certain parts of the brain may lead to problems with consciousness. Letting '
 'go of material possessions and external desires is crucial for spiritual '
 'growth, and our choices determine which principle of fate and free will we '
 'empower. Ultimately, our goal is to find our own happiness.')


In [20]:
# refine
# https://python.langchain.com/en/latest/reference/modules/text_splitter.html

from langchain import text_splitter
from langchain.docstore.document import Document
from langchain import PromptTemplate                 

# bullet_point_prompt_template = """Write notes about the following transcription. Use bullet points in complete sentences. 
# Use * (asterisk followed by a space) for the point. Include all details such as titles, citations, dates and so on.
# Any text that resembles Roko should be spelled Roko. Some common misspellings include Rocko and Rocco. Please make
# sure to spell it correctly.
bullet_point_prompt_template = """Do summarization for given transcription. provide language in Korean.

{text}

NOTES:
"""

bullet_point_prompt = PromptTemplate(template=bullet_point_prompt_template, input_variables=["text"])
paragraphs = text_splitter.TokenTextSplitter().split_text(transcript)
docs = [Document(page_content=t) for t in paragraphs]

llmc = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")
chain = load_summarize_chain(llmc, chain_type="refine", return_intermediate_steps=True, refine_prompt=bullet_point_prompt)

res = chain({"input_documents": docs}, return_only_outputs=True)

In [2]:
from langchain import text_splitter
doc_paragraphs = text_splitter.TextSplitter().create_documents([transcript])
doc_paragraphs

TypeError: Can't instantiate abstract class TextSplitter with abstract method split_text

In [26]:
docs

[Document(page_content='여기 두 존재가 있습니다. 하나는 인공지능이고요. 하나는 인간입니다. 겉으로 보기에는 완전히 똑같은 두 존재의 정체성을 가르는 것은 단 한 가지입니다. 그것은 바로 생각하고 감정을 느낄 수 있는 힘. 의식입니다. 우리는 이 의식이라는 게 그냥 뇌라는 장기에서 뚝딱 만들어진 무산물이라고 배워왔습니다. 하지만 최첨단 현대 물리학계에서 기존의 독념을 완전히 뒤집어버리는 관념이 제기되고 있습니다. 현재 생존해 있는 과학자들 중에서 가장 위대한 물리학자로 칭송받고 있는 과학자 2020년 노벨 물리학상을 수상한 로저 펜 로즈는 이렇게 말했습니다. 인간의 의식은 뇌라는 기계장치 따위에서는 만들어질 수 없는 그 무언가이다. 만약 어떤 사람이 일시적으로 사망하게 되면 그 사람의 두뇌 속 미세소관이라는 부분에 있던 양자 정보가 우주로 흘러들어가게 된다. 그러나 죽음 근처까지 갔던 사람이 다시 살아난다면 두뇌에서 유출된 양자 정보가 다시 두뇌 속으로 돌아가게 된다. 이때가 바로 임사체험을 하게 되는 경우다. 만약 그가 되살아나지 않고 완전히 죽게 되더라도 그의 두뇌 속에 있던 양자 정보는 영혼으로써 육체 바깥에서 영원히 존재할 수 있다. 왜냐? 양자 정보는 절대로 파괴될 수 없기 때문이다. 이게 도대체 무슨 말인지 아리가리 하신 분들을 위해서 양자 의식 이론의 핵심만 간단하게 요약해보겠습니다. 인간의 의식은 양자 물질로 구성되어 있기 때문에 양자 에너지적 성질을 가진다. 의식은 양자 에너지의 형태로 두뇌 속에 미세소관이라는 부분에 담겨 있다가 육체가 죽는 순간 몸 밖으로 빠져나간다. 그렇게 육체 바깥으로 빠져나간 의식 에너지는 인간의 육체를 떠나서도 영원히 존재할 수 있다. 아니 그러니까 인류가 오래전부터 영혼이라고 불러봤던 것이 비로소 현대 물리학인 양자 역학으로도 설명이 가능하게 된 거예요. 자 의식이 양자적 성질을 가지는 에너지라면 이 의식 에너지는 존재하는 모든 에너지에 예외 없이 적용되는 에너지 보존법칙에 영향을 받게 됩니다. 

In [23]:
res

{'intermediate_steps': ['The concept of quantum consciousness posits that consciousness is not just a product of the brain, but a form of quantum energy that can exist outside of the physical body. According to the law of energy conservation, this energy cannot be destroyed and will continue to exist even after death. This concept is not only accepted in Buddhism, but also in Christianity, where the idea of immortality of the soul is recognized. The principle of karma is also emphasized in both religions as a way to release negative energy and achieve spiritual salvation. However, the concept of reincarnation was removed from Christian teachings due to political and cultural reasons.',
  '양자의식이론을 통해 귀신, 임사체형, 육체 이탈 등 초자연적인 현상이 설명 가능하다. 의식 에너지는 차원을 초월하는 존재이지만, 육체에 기뜩이면 3차원적으로 제한된 사고를 하게 된다. 따라서, 육체의 힘을 빼는 수행을 하면 의식 에너지의 작용력이 강해진다는 것을 수천 년 전부터 이미 알고 있었던 수행인들이 있었다. 환생 역시 양자의식이론으로 설명 가능하며, 환생 사례가 많이 조사되고 있다. 과학과 영성은 분리되어 있어서는 안 된다. 어린이들이 전생을 기억한 것은 뇌의 기능과 관련되어 있기 때문에 시간이 지나면서 잊혀진다.',
  '3-

In [25]:
pprint(res['output_text'].replace('. ', '. \n'))

('이 글에서는 놓음을 통해 집착에서 벗어나 현재에 집중하고 나의 내면과 감정 상태를 관찰하는 것이 중요하다는 것을 이야기합니다. \n'
 '카르마와 자유의지는 모순적인 원리이지만 둘 다 동시에 존재하고 있으며, 각자만의 선택에 따라 운명에 힘을 실어줄 것이냐 자유의지에 힘을 '
 '실어줄 것이냐 결정할 수 있다는 것을 말합니다. \n'
 '이용과 해소 중에서 마음이 끌리는 쪽을 선택하면서 각자의 행복을 찾아가면 된다는 것을 마지막으로 언급하고 있습니다.')


In [ ]:
# map_reduce 는 안됨
# map_reduce와 refine의 기능 차이를 알아야함

# https://python.langchain.com/en/latest/reference/modules/text_splitter.html

from langchain import text_splitter
from langchain.docstore.document import Document
from langchain import PromptTemplate                 

# bullet_point_prompt_template = """Write notes about the following transcription. Use bullet points in complete sentences. 
# Use * (asterisk followed by a space) for the point. Include all details such as titles, citations, dates and so on.
# Any text that resembles Roko should be spelled Roko. Some common misspellings include Rocko and Rocco. Please make
# sure to spell it correctly.
bullet_point_prompt_template = """Do summarization for given transcription. provide language in Korean.

{text}

NOTES:
"""

bullet_point_prompt = PromptTemplate(template=bullet_point_prompt_template, input_variables=["text"])
paragraphs = text_splitter.TokenTextSplitter().split_text(transcript)
docs = [Document(page_content=t) for t in paragraphs]

llmc = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")
chain = load_summarize_chain(llmc, chain_type="map_reduce", return_intermediate_steps=True, map_reduce_prompt=bullet_point_prompt)

print(chain({"input_documents": docs}, return_only_outputs=True))

In [ ]:
# https://github.com/hwchase17/langchain/issues/833

from langchain import PromptTemplate

bullet_point_prompt_template = """Write notes about the following transcription. Use bullet points in complete sentences. 
Use * (asterisk followed by a space) for the point. Include all details such as titles, citations, dates and so on.
Any text that resembles Roko should be spelled Roko. Some common misspellings include Rocko and Rocco. Please make
sure to spell it correctly.

{text}

NOTES:
"""

bullet_point_prompt = PromptTemplate(template=bullet_point_prompt_template, input_variables=["text"])
def generate_summary(audio_id):
    print(f"Generating summary for transcription {audio_id}")
    audio = db.getAudio(audio_id)
    text_splitter = NLTKTextSplitter(chunk_size=2000)
    texts = text_splitter.split_text(audio['transcription'])
    docs = [Document(page_content=t) for t in texts]
    chain = load_summarize_chain(llm, chain_type="refine", 
    return_intermediate_steps=True, refine_prompt=bullet_point_prompt)
    print(chain({"input_documents": docs}, return_only_outputs=True))

In [ ]:
summary_chain_cgpt35.run(transcript)

In [ ]:
pprint(summarize_document_chain_cgpt35.run(transcript))

### How much?

- text-davinci, 6 requests
- 101,611 prompt + 8,930 completion = 110,541 tokens
- $0.47 == ₩611

## Question Answering 질문 답변

1~3분 소요

In [41]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo") # gpt-3.5-turbo, gpt-4

qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [42]:
qa_document_chain.run(
    input_document=raw_text,
    question="싱클레어를 괴롭힌 사람은?")

'주어진 텍스트에는 싱클레어를 괴롭힌 사람에 대한 구체적인 정보가 제공되지 않았습니다. 따라서, 이 질문에 대한 정확한 답변을 제공할 수 없습니다.'

In [ ]:
qa_document_chain.run(
    input_document=raw_text,
    question="싱클레어는 데미안을 어디서 어떻게 만났지?")

'제공된 텍스트에서는 싱클레어가 데미안을 만난 경위에 대한 구체적인 정보가 제공되지 않습니다. 따라서 이 질문에 대한 답변을 제공할 수 없습니다.'

In [ ]:
qa_document_chain.run(
    input_document=raw_text,
    question="데미안의 외모를 묘사해봐")

'저는 주어진 텍스트에서 데미안의 외모에 대한 구체적인 묘사를 찾을 수 없었습니다. 따라서 데미안의 외모에 대한 정보를 제공할 수 없습니다.'

### GPT-4

11분 소요

- gpt-4-0314, 28 requests
    - 29,503 prompt + 1,111 completion = 30,614 tokens tokens
- gpt-4-0314, 42 requests
    - 44,041 prompt + 1,784 completion = 45,825 tokens
- gpt-4-0314, 10 requests
    - 10,652 prompt + 865 completion = 11,517 tokens


87,956 tokens total == $5.28

> $0.06 / 1K tokens

In [ ]:
model = ChatOpenAI(model="gpt-4") # gpt-3.5-turbo, gpt-4

qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

qa_document_chain.run(
    input_document=raw_text,
    question="데미안의 외모를 묘사해봐")

'데미안의 외모는 다음과 같이 묘사되어 있습니다: 근접한 갈색 머리카락, 반여성적인 입, 이상한 밝기를 가진 강한 이마, 그리고 넓게 벌어진 녹색 눈동자로, 오른쪽 눈이 왼쪽 눈보다 약간 높게 위치해 있습니다. 또한, 데미안은 군복을 입고 은색 회색 망토를 입은 모습이 기묘하고 이상한 것처럼 묘사되어 있습니다.'

## 엑셀, CSV 검색, Aggregation

In [45]:
import pandas as pd

df = pd.read_csv("https://github.com/kairess/toy-datasets/raw/master/titanic.csv")

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [46]:
from langchain.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: len(df)
Observation: 891
Thought: I now know the final answer
Final Answer: There are 891 rows in the dataframe.

> Finished chain.


'There are 891 rows in the dataframe.'

In [ ]:
agent.run("행이 몇 개지?")



> Entering new AgentExecutor chain...
Thought: 데이터 프레임의 크기를 알아보자
Action: python_repl_ast
Action Input: df.shape
Observation: (891, 12)
Thought: 행의 개지는 첫 번째 숫자임
Final Answer: 891개

> Finished chain.


'891개'

In [47]:
agent.run("승객들의 평균 연령은?")



> Entering new AgentExecutor chain...
Thought: 나는 이 데이터 프레임에서 평균 연령을 찾아야한다.
Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 29.69911764705882
Thought: 나는 이제 최종 답을 알겠다.
Final Answer: 승객들의 평균 연령은 29.7세입니다.

> Finished chain.


'승객들의 평균 연령은 29.7세입니다.'

In [ ]:
agent.run("남성과 여성의 비율은?")



> Entering new AgentExecutor chain...
Thought: 남성과 여성의 수를 세야한다.
Action: python_repl_ast
Action Input: df['Sex'].value_counts()
Observation: male      577
female    314
Name: Sex, dtype: int64
Thought: 남성과 여성의 비율을 계산해야한다.
Action: python_repl_ast
Action Input: df['Sex'].value_counts() / df['Sex'].count()
Observation: male      0.647587
female    0.352413
Name: Sex, dtype: float64
Thought: 남성과 여성의 비율을 알았다.
Final Answer: 남성의 비율은 0.647587, 여성의 비율은 0.352413입니다.

> Finished chain.


'남성의 비율은 0.647587, 여성의 비율은 0.352413입니다.'

In [ ]:
agent.run("객실 등급과 성별에 따른 생존자 수를 계산해줘")



> Entering new AgentExecutor chain...
Thought: 먼저 객실 등급과 성별로 그룹화해야 할 것 같다.
Action: python_repl_ast
Action Input: df.groupby(['Pclass', 'Sex'])['Survived'].sum()
Observation: Pclass  Sex   
1       female    91
        male      45
2       female    70
        male      17
3       female    72
        male      47
Name: Survived, dtype: int64
Thought: 이제 생존자 수를 계산할 수 있다.
Final Answer: 객실 등급과 성별에 따른 생존자 수는 각각 91명, 45명, 70명, 17명, 72명, 47명입니다.

> Finished chain.


'객실 등급과 성별에 따른 생존자 수는 각각 91명, 45명, 70명, 17명, 72명, 47명입니다.'